<a href="https://colab.research.google.com/github/martinpius/SEQUENCE-MODELS-FINAL/blob/main/NMT__WITH_ATTENTION_STEP_BY_STEP_CLEAN_VERSION.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f">>>> You are on Google CoLaB with Pytorch version {torch.__version__}")
except Exception as e:
  print(f">>>> {type(e)}: {e}\n>>>> please correct {type(e)} and reload your Google drive")
  COLAB = False
if torch.cuda.is_available():
  device = torch.device("cuda")
else:
  device = torch.device("cpu")
def time_fmt(t: float = 123.918)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"hrs: {h} min: {m:>02} sec: {s:>05.2f}"
print(f">>>> testing the time formating function.........\n>>>> time elapsed\t{time_fmt()}")

Mounted at /content/drive
>>>> You are on Google CoLaB with Pytorch version 1.9.0+cu102
>>>> testing the time formating function.........
>>>> time elapsed	hrs: 0 min: 02 sec: 03.00


In [2]:
import torch, spacy, time, datetime, random, math, sys
import numpy as np
from tensorflow import summary
from torchtext.legacy.datasets import Multi30k
from torchtext.legacy.data import BucketIterator, Field
from torch import nn
from torch import optim
from tqdm import tqdm
%load_ext tensorboard


In [3]:
!python -m spacy download en_core_web_sm
!python -m spacy download de_core_news_sm
en_tkn = spacy.load("en_core_web_sm")
de_tkn = spacy.load("de_core_news_sm")


     |████████████████████████████████| 12.0 MB 6.9 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
     |████████████████████████████████| 14.9 MB 7.7 MB/s 
✔ Download and installation successful
You can now load the model via spacy.load('de_core_news_sm')


In [4]:
def en_tokenizer(text):
  return [tok.text for tok in en_tkn.tokenizer(text)]
def de_tokenizer(text):
  return [tok.text for tok in de_tkn.tokenizer(text)]
  

In [5]:
english = Field(tokenize = en_tokenizer, init_token = "sos", eos_token = "eos", lower = True)
germany = Field(tokenize = de_tokenizer, init_token = "sos", eos_token = "eos", lower = True)
tic = time.time()
train_data, valid_data, test_data = Multi30k.splits(fields = (germany, english), exts = (".de", ".en"))
print(f">>>> total train examples: {len(train_data.examples)}\n>>>> total valid examples: {len(valid_data.examples)}\n>>>> total test examples: {len(test_data.examples)}")
print(f">>>> first paired example in a train set: {vars(train_data.examples[0])}")
toc = time.time()
print(f">>>> total time for loading and preprocess the data: {time_fmt(toc - tic)}")

downloading training.tar.gz


training.tar.gz: 100%|██████████| 1.21M/1.21M [00:01<00:00, 861kB/s]


downloading validation.tar.gz


validation.tar.gz: 100%|██████████| 46.3k/46.3k [00:00<00:00, 166kB/s]


downloading mmt_task1_test2016.tar.gz


mmt_task1_test2016.tar.gz: 100%|██████████| 66.2k/66.2k [00:00<00:00, 158kB/s]


>>>> total train examples: 29000
>>>> total valid examples: 1014
>>>> total test examples: 1000
>>>> first paired example in a train set: {'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}
>>>> total time for loading and preprocess the data: hrs: 0 min: 00 sec: 09.00


In [6]:
english.build_vocab(train_data, min_freq = 2)
germany.build_vocab(train_data, min_freq = 2)

In [7]:
batch_size = 128
train_iter, valid_iter, test_iter = BucketIterator.splits(datasets = (train_data, valid_data, test_data),
                                                   batch_size = batch_size, device = device)

In [9]:
for i, batch in enumerate(train_iter):
  print(batch.src.shape)
  sys.exit()

torch.Size([25, 128])


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [10]:
class Encoder(nn.Module):
  def __init__(self, enc_input_dim, enc_embd_dim, enc_hidden_dim, dec_hidden_dim, dropout):
    super(Encoder, self).__init__()
    self.dropout = nn.Dropout(dropout)
    self.enc_embedding = nn.Embedding(num_embeddings = enc_input_dim, embedding_dim = enc_embd_dim)
    self.enc_gru = nn.GRU(input_size = enc_embd_dim, hidden_size = enc_hidden_dim, bidirectional = True)
    self.fc = nn.Linear(in_features = 2* enc_hidden_dim, out_features = dec_hidden_dim)
  
  def forward(self, enc_inputs):
    """
    enc_input: shape == [enc_seq_len, batch_size]
    enc_out: shape == [enc_seq_len, batch_size, 2* enc_hidden_dim]
    enc_hidden: shape == [batch_size, enc_hidden_dim]
    enc_hidden == cat[enc_hidden[forward], enc_hidden[backward]]
    """
    enc_embeded = self.dropout(self.enc_embedding(enc_inputs)) # shape == [enc_seq_len, batch_size, enc_embd_dim]
    enc_out, enc_hidden = self.enc_gru(enc_embeded)
    hidden = torch.tanh(self.fc(torch.cat((enc_hidden[-2,:,:], enc_hidden[-1,:,:]), dim = 1))) # shape == [batch_size, dec_hidden_dim]
    return enc_out, hidden
    

In [11]:
# testing the class
for i, batch in enumerate(tqdm(train_iter)):
  enc_input = batch.src
  encoder = Encoder(len(germany.vocab), 300, 512, 512, 0.5).to(device = device)
  enc_out, enc_hidden = encoder(enc_input)
  print(f"\n>>>> enc_hidden_shape: {enc_hidden.shape}\n>>>> enc_out_shape: {enc_out.shape}")
  sys.exit()

  0%|          | 0/227 [00:00<?, ?it/s]


>>>> enc_hidden_shape: torch.Size([128, 512])
>>>> enc_out_shape: torch.Size([27, 128, 1024])


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [12]:
class Attention(nn.Module):
  def __init__(self, enc_hidden_dim, dec_hidden_dim):
    super(Attention, self).__init__()
    self.attn = nn.Linear(in_features = 2*enc_hidden_dim + dec_hidden_dim, out_features = dec_hidden_dim)
    self.v = nn.Linear(in_features = dec_hidden_dim, out_features = 1, bias = False)
  
  def forward(self, enc_out, dec_hidden):
    """
    enc_out: shape == [enc_seq_len, batch_size, 2*enc_hidden_dim]
    dec_hidden: shape == [batch_size, dec_hidden_dim]
    # we combine enc_out, dec_hidden and compute attention weights
    """
    batch_size = enc_out.shape[1]
    enc_seq_len = enc_out.shape[0]
    dec_hidden = dec_hidden.unsqueeze(1).repeat(1, enc_seq_len, 1) # shape == [batch_size, enc_seq_len, dec_hidden]
    enc_out = enc_out.permute(1,0,2) # shape == [batch_size, enc_seq_len, 2*enc_hidden_dim]
    energy = torch.tanh(torch.cat((enc_out, dec_hidden), dim = 2)) # shape == [batch_size, enc_seq_len, 2*enc_hidden_dim + dec_hidden_dim]
    attention = self.attn(energy)
    attention = self.v(attention).squeeze(2) # shape == [batch_size, enc_seq_len]
    return nn.functional.softmax(attention, dim = 1) # shape == [batch_size, enc_seq_len]

In [13]:
# testing

In [14]:
for i, batch in enumerate(tqdm(train_iter)):
  input = batch.src
  encoder = Encoder(len(germany.vocab), 300, 512, 512, 0.5).to(device = device)
  enc_out, enc_hidden = encoder(input)
  attention = Attention(512, 512).to(device = device)
  test_attention = attention(enc_out, enc_hidden)
  print(f"\n>>>> attention_shape: {test_attention.shape}")
  sys.exit()

  0%|          | 0/227 [00:00<?, ?it/s]


>>>> attention_shape: torch.Size([128, 28])


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [15]:
# Decoder
class Decoder(nn.Module):
  def __init__(self, dec_out_dim, dec_embd_dim, dec_hidden_dim, enc_hidden_dim, attention, dropout):
    super(Decoder, self).__init__()
    self.dropout = nn.Dropout(dropout)
    self.attention = attention
    self.dec_out_dim = dec_out_dim
    self.dec_embd = nn.Embedding(num_embeddings = dec_out_dim, embedding_dim = dec_embd_dim)
    self.dec_gru = nn.GRU(input_size = (2*enc_hidden_dim + dec_embd_dim), hidden_size = dec_hidden_dim)
    self.fc = nn.Linear(in_features = (2*enc_hidden_dim + dec_embd_dim + dec_hidden_dim), out_features = dec_out_dim)

  def forward(self, dec_input, dec_hidden, enc_out):
    """
    dec_input: shape == [batch_size] # we enter one token at a time
    dec_hidden: shape == [batch_size, dec_hidden_dim]
    enc_out: shape == [enc_seq_len, batch_size, 2*enc_hidden_dim]
    ## we need to compute the predictions using the context vector derived
    ## from decoder hidden + enc_out and attention
    dec_out: shape == [dec_seq_len, batch_size, dec_hidden_dim]
    hidden: shape ==m[batch_size, dec_hidden_dim]

    """
    dec_input = dec_input.unsqueeze(0) # shape == [1, batch_size]
    dec_embeded = self.dropout(self.dec_embd(dec_input)) # shape == [1, batch_size, dec_embd_dim]
    a = self.attention(enc_out, dec_hidden) # shape == [batch_size, enc_seq_len]
    a = a.unsqueeze(1) # shape ==  [batch_size, 1, enc_seq_len]
    enc_out = enc_out.permute(1, 0, 2) # shape == [batch_size, enc_seq_len, 2*enc_hidden_dim]
    weighted = torch.bmm(a, enc_out) # shape == [batch_size, 1, 2*enc_hidden_dim] 
    weighted = weighted.permute(1, 0, 2) # shape == [1, batch_size, 2*enc_hidden_dim]
    gru_input = torch.cat((weighted, dec_embeded), dim = 2) # shape == [1, batch_size, 2*enc_hidden_dim + dec_embd_dim]
    dec_out, hidden = self.dec_gru(gru_input, dec_hidden.unsqueeze(0))
    weighted = weighted.squeeze(0) # shape == [batch_size, 2*enc_hidden_dim]
    dec_embeded = dec_embeded.squeeze(0) # shape == [batch_size, dec_embedded_dim]
    dec_out = dec_out.squeeze(0) # shape == [batch_size, dec_hidden_dim]
    fc_input = torch.cat((weighted, dec_embeded, dec_out), dim = 1) # shape == [batch_size, 2*enc_hidden_dim + dec_embeded_dim+ dec_hidden]
    preds = self.fc(fc_input) # shape == [batch_size, dec_out_dim]
    return preds, hidden.squeeze(0)


In [16]:
# testing the decoder's class:
for i, batch in enumerate(tqdm(train_iter)):
  enc_input = batch.src
  dec_input = batch.trg
  encoder = Encoder(len(germany.vocab), 300, 512, 512, 0.5).to(device = device)
  enc_out, enc_hidden = encoder(enc_input)
  attention = Attention(512, 512)
  decoder = Decoder(len(english.vocab), 300, 512, 512, attention,0.5).to(device = device)
  preds, dec_hidden = decoder(dec_input[i], enc_hidden, enc_out)
  print(f"\n>>>> predictions_shape: {preds.shape}\n>>>> dec_hidden_shape: {dec_hidden.shape}")
  sys.exit()

  0%|          | 0/227 [00:00<?, ?it/s]


>>>> predictions_shape: torch.Size([128, 5893])
>>>> dec_hidden_shape: torch.Size([128, 512])


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [17]:
# Model class:
class Final_Model(nn.Module):
  def __init__(self, encoder, decoder, device):
    super(Final_Model, self).__init__()
    self.encoder = encoder
    self.decoder = decoder
    self.device = device
  
  def forward(self, input, target, tfl_ratio = 0.5):
    """
    input: shape == [seq_len, batch_size]
    target: shape == [seq_len, batch_size]
    tfl_ratio: teacher force learning ratio
    outputs shape == [dec_seq_len, batch_size, dec_voc_size]
    """
    batch_size = input.shape[1]
    target_len = target.shape[0]
    target_voc_size = self.decoder.dec_out_dim
    outputs = torch.zeros(size = (target_len, batch_size, target_voc_size)).to(device = self.device)
    init_token = target[0,:] # select the first token in a decoder's sequence ("sos")
    enc_out, enc_hidden = self.encoder(input) # run the encoder network
    for t in range(1, target_len):
      preds, dec_hidden = self.decoder(init_token, enc_hidden, enc_out)
      outputs[t] = preds
      best_guess = preds.argmax(1)
      if tfl_ratio < random.random():
        input = best_guess
      else:
        input = target[t]
    
    return outputs # shape == [target_len, batch_size, target_voc_size]


In [18]:
# testing:
for i, batch in enumerate(tqdm(train_iter)):
  input = batch.src
  target = batch.trg
  encoder = Encoder(len(germany.vocab), 300, 512, 512, 0.5).to(device = device)
  attention = Attention(512, 512)
  decoder = Decoder(len(english.vocab), 300, 512, 512,attention, 0.5).to(device = device)
  model = Final_Model(encoder, decoder, device).to(device = device)
  predictions = model(input, target)
  print(f"\n>>>> predictions_shape: {predictions.shape}")
  sys.exit()

  0%|          | 0/227 [00:00<?, ?it/s]


>>>> predictions_shape: torch.Size([26, 128, 5893])


SystemExit: ignored

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2890: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [19]:
# instantiating the model class:
enc_input_dim = len(germany.vocab)
dec_input_dim = len(english.vocab)
enc_hidden_dim = 512
dec_hidden_dim = 512
enc_embd_dim = 300
dec_embd_dim = 300
dropout = 0.5
attention = Attention(enc_hidden_dim, dec_hidden_dim).to(device = device)
encoder = Encoder(enc_input_dim, enc_embd_dim, enc_hidden_dim, dec_hidden_dim, dropout).to(device = device)
decoder = Decoder(dec_input_dim, dec_embd_dim, dec_hidden_dim, enc_hidden_dim, attention, dropout).to(device = device)
model = Final_Model(encoder, decoder, device).to(device = device)


In [20]:
# initialize the weights to random normal (0.00, 0.001) and the biasses to 0
def initializer(model):
  for name, par in model.named_parameters():
    if "weight" in name:
      nn.init.normal_(par.data, mean = 0.00, std = 0.01)
    else:
      nn.init.constant_(par.data, 0)
model.apply(initializer)

Final_Model(
  (encoder): Encoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (enc_embedding): Embedding(7855, 300)
    (enc_gru): GRU(300, 512, bidirectional=True)
    (fc): Linear(in_features=1024, out_features=512, bias=True)
  )
  (decoder): Decoder(
    (dropout): Dropout(p=0.5, inplace=False)
    (attention): Attention(
      (attn): Linear(in_features=1536, out_features=512, bias=True)
      (v): Linear(in_features=512, out_features=1, bias=False)
    )
    (dec_embd): Embedding(5893, 300)
    (dec_gru): GRU(1324, 512)
    (fc): Linear(in_features=1836, out_features=5893, bias=True)
  )
)

In [21]:
# parameters conter
def par_counter(model):
  return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f"\n>>>> total number of learnable weights in the model: {par_counter(model):,} parameters")


>>>> total number of learnable weights in the model: 21,585,873 parameters


In [22]:
# training hyper-parameters and tensorboard setup:
EPOCHS = 100
learning_rate = 1e-3
current_time = datetime.datetime.now().timestamp()
train_path = "logs/tensorboard/train_mtl_attn/" + str(current_time)
valid_path = "logs/tensorboard/valid_mtl_attn/" + str(current_time)
train_writer = summary.create_file_writer(logdir = train_path)
valid_writer = summary.create_file_writer(logdir = valid_path)


In [23]:
# get the loss and the optimizer objects

In [24]:
padding_idx = english.vocab.stoi["<pad>"]
loss_obj = nn.CrossEntropyLoss(ignore_index = padding_idx)
optimizer = optim.Adam(params = model.parameters(), lr = learning_rate)

In [25]:
# the train loop
def training_loop(model, iterator, loss_obj, optimizer, clip):
  model.train()
  step = 0
  loss_per_epoch = 0
  for i, batch in enumerate(tqdm(iterator)):
    input = batch.src.to(device = device)
    target = batch.trg.to(device = device)
    optimizer.zero_grad() # initialize the slopes to zeros
    preds = model(input, target) # shape == [target_len, batch_size, target_voc_size]
    out_dim = preds.shape[-1] # grab the target_voc_size
    # reshape to suit the cross-entropy loss requirement
    preds = preds[1:].view(-1, out_dim) # shape = [(target_len - 1) * batch_size, target_voc_size]
    target = target[1:].view(-1) # shape == [(target_len - 1) * batch_size]
    train_loss = loss_obj(preds, target)
    train_loss.backward() # compute the gradients
    nn.utils.clip_grad_norm_(parameters = model.parameters(), max_norm = clip)
    optimizer.step() # gradient descent with adam optimizer
    loss_per_epoch+=train_loss.item()
    if i % 100 == 0:
      with train_writer.as_default():
        summary.scalar("train_loss", train_loss.item(), step = step)
        step += 1
  return loss_per_epoch / len(iterator)



In [26]:
# validation loop
def valid_loop(model, iterator, loss_obj):
  model.eval()
  step = 0
  loss_per_epoch = 0
  for i, batch in enumerate(tqdm(iterator)):
    input = batch.src
    target = batch.trg
    preds = model(input, target, 0) # no teacher-force technique at this stage
    out_dim = preds.shape[-1]
    preds = preds[1:].view(-1, out_dim)
    target = target[1:].view(-1)
    valid_loss = loss_obj(preds, target)
    loss_per_epoch += valid_loss.item()
    if i % 100 == 100:
      with valid_writer.as_default():
        summary.scalar("valid_loss", valid_loss.item(), step = step)
        step += 1
  
  return loss_per_epoch / len(iterator)
  

In [27]:
clip = 1
best_valid_loss = float("inf")
tic = time.time()
for epoch in range(EPOCHS):
  print(f"\n>>>> training starts for epoch {epoch + 1}\n>>>> please wait while the model is training....................")
  train_loss = training_loop(model, train_iter, loss_obj, optimizer, clip)
  valid_loss = valid_loop(model, valid_iter, loss_obj)
  if valid_loss < best_valid_loss:
    torch.save(model.state_dict(), "mtl_attn.pt")
    best_valid_loss = valid_loss
  print(f"\n>>>> end of training for epoch {epoch + 1}, train loss: {train_loss:.4f}, train PPL: {math.exp(train_loss):7.4f}")
  print(f">>>> end of validation for epoch {epoch + 1}, valid loss: {valid_loss:.4f}, valid PPL: {math.exp(valid_loss):7.4f}")
toc = time.time()
print(f"\n>>>> end of training for {EPOCHS} epochs: total time elapsed is {time_fmt(toc - tic)}")

%tensorboard -logdir logs/tensorboard


>>>> training starts for epoch 1
>>>> please wait while the model is training....................


100%|██████████| 8/8 [00:00<00:00, 13.26it/s]



>>>> end of training for epoch 1, train loss: 5.5112, train PPL: 247.4592
>>>> end of validation for epoch 1, valid loss: 5.1999, valid PPL: 181.2520

>>>> training starts for epoch 2
>>>> please wait while the model is training....................


100%|██████████| 8/8 [00:00<00:00, 13.16it/s]



>>>> end of training for epoch 2, train loss: 5.0895, train PPL: 162.3007
>>>> end of validation for epoch 2, valid loss: 4.8191, valid PPL: 123.8591

>>>> training starts for epoch 3
>>>> please wait while the model is training....................


...

100%|██████████| 8/8 [00:00<00:00, 12.79it/s]



>>>> end of training for epoch 99, train loss: 2.8177, train PPL: 16.7378
>>>> end of validation for epoch 99, valid loss: 4.3337, valid PPL: 76.2265

>>>> training starts for epoch 100
>>>> please wait while the model is training....................


100%|██████████| 8/8 [00:00<00:00, 12.81it/s]


>>>> end of training for epoch 100, train loss: 2.8157, train PPL: 16.7048
>>>> end of validation for epoch 100, valid loss: 4.3327, valid PPL: 76.1484

>>>> end of training for 100 epochs: total time elapsed is hrs: 2 min: 28 sec: 49.00


ERROR: Failed to launch TensorBoard (exited with 2).
Contents of stderr:
usage: tensorboard [-h] [--helpfull] [--logdir PATH] [--logdir_spec PATH_SPEC]
                   [--host ADDR] [--bind_all] [--port PORT]
                   [--reuse_port BOOL] [--load_fast {false,auto,true}]
                   [--extra_data_server_flags EXTRA_DATA_SERVER_FLAGS]
                   [--grpc_creds_type {local,ssl,ssl_dev}]
                   [--grpc_data_provider PORT] [--purge_orphaned_data BOOL]
                   [--db URI] [--db_import] [--inspect] [--version_tb]
                   [--tag TAG] [--event_file PATH] [--path_prefix PATH]
                   [--window_title TEXT] [--max_reload_threads COUNT]
                   [--reload_interval SECONDS] [--reload_task TYPE]
                   [--reload_multifile BOOL]
                   [--reload_multifile_inactive_secs SECONDS]
                   [--generic_data TYPE]
                   [--samples_per_plugin SAMPLES_PER_PLUGIN]
                   [-